In [8]:
import os
from os import path
import torch
from torch import nn
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torchvision.models as models
from torchvision.transforms import ToTensor, ToPILImage, Lambda
import matplotlib.pyplot as plt
import math
import autolens as al

from PIL import Image

import numpy as np
import pandas as pd
import json
from torchvision.io import read_image

In [20]:
def ell_comps_from(axis_ratio, angle):
    """
    Convert an input axis ratio (0.0 > q > 1.0) and rotation position angle defined counter clockwise from the
    positive x-axis(0.0 > angle > 180) to the (y,x) ellipitical components e1 and e2.

    Parameters
    ----------
    axis_ratio
        Ratio of light profiles ellipse's minor and major axes (b/a).
    angle
        Rotation angle of light profile counter-clockwise from positive x-axis.
    """
    angle *= np.pi / 180.0
    fac = (1 - axis_ratio) / (1 + axis_ratio)
    ellip_y = fac * np.sin(2 * angle)
    ellip_x = fac * np.cos(2 * angle)
    return (ellip_y, ellip_x)

In [6]:
root = "__local/data/Cosmosim/slacs/raw"
lens_path = os.listdir(root)

In [34]:
profiles = []
for lens in lens_path:
    data = {}
    directory = path.join(root, lens, 'info.json')
    ser = pd.read_json(directory, typ='series')
    ell_comps = ell_comps_from(ser.axis_ratio, ser.position_angle)
    data['name'] = ser.data_name
    data['rdsf_l'] = ser.redshift_lens
    data['r_ein'] = ser.einstein_radius
    data['ellx_l'] = ell_comps[0]
    data['ellx_y'] = ell_comps[1]
    data['rdsf_s'] = ser.redshift_source
    data['r_eff_s'] = ser.r_eff
    profiles.append(data)

In [35]:
df = pd.DataFrame(profiles)

In [36]:
df

,name,rdsf_l,r_ein,ellx_l,ellx_y,rdsf_s,r_eff_s
0,J2238-0754,0.137,1.27,-0.080797,0.006785,0.713,2.33
1,J0728+3835,0.206,1.25,0.057133,-0.057533,0.688,1.78
2,J1630+4520,0.248,1.78,0.034969,-0.060083,0.793,1.96
3,J1416+5136,0.299,1.37,0.018699,-0.024635,0.811,1.43
4,J1250+0523,0.232,1.13,-0.020189,-0.002981,0.795,1.81
5,J1205+4910,0.215,1.22,-0.128642,0.120802,0.481,2.59
6,J1525+3327,0.358,1.31,-0.324406,-0.007928,0.717,2.90
7,J1142+1001,0.222,0.98,-0.030244,-0.087835,0.504,1.91
8,J0946+1006,0.222,1.38,-0.069694,0.078498,0.609,2.35
9,J1213+6708,0.123,1.42,0.045037,0.081249,0.640,3.23


In [37]:
df.to_csv('__local/data/Cosmosim/slacs/labels.csv')